Fails uglily (:P) on things that cross the dateline or have far-away parts. no idea what's wrong with china...

In [1]:
import fiona
from shapely.geometry import shape, mapping, Point, GeometryCollection
from fiona.crs import from_string
from fiona.transform import transform_geom

import ipywidgets as widgets
from IPython.display import display,clear_output
from shapely.geometry import *

from IPython.core.display import HTML
HTML("""
<style>
div.output_svg div {
    width: 100% !important;
    height: 100% !important;
}
</style>
""")

In [2]:
with fiona.open("data/ne_110m_admin_0_countries.shp") as source:
    features = list(source)
    source_crs = source.crs

names = sorted([f['properties']['name'] for f in features])

In [3]:
def feature_by_attribute_value(features, attribute, value):
    # returns the FIRST matching feature
    return [feature for feature in features if feature['properties'][attribute] == value][0]

In [10]:
def project_locally(feature):
    geometry = shape(feature['geometry'])
    lat_0, lon_0 = geometry.centroid.xy
    target_crs = from_string("+proj=aeqd +R=6371000 +lat_0={} +lon_0={}".format(lat_0, lon_0))
    projected_geom = shape(transform_geom(source_crs, target_crs, mapping(geometry)))
    projected_geom = projected_geom.buffer(0)  # buffer to fix self-intersections
    return projected_geom

In [11]:
def explore_local_projections(name):
    feature = feature_by_attribute_value(features, "name", name)
    return project_locally(feature)

In [12]:
widgets.interactive(explore_local_projections, name=names)

## But let's make something to visually compare before and after

In [18]:
def update_displays(_):
    name = dd.value
    feature = feature_by_attribute_value(features, "name", name)
    
    with outleft:
        g = shape(feature['geometry'])
        clear_output(wait=True)
        display(g)
        
    with outright:
        g = project_locally(feature)
        clear_output(wait=True)
        display(g)

outleft=widgets.Output()
outright=widgets.Output()

dd = widgets.Dropdown(
    options=names,
    description='Country:',
)
dd.observe(update_displays)

displays = widgets.HBox(children=(outleft, outright))
dashboard = widgets.VBox(children=(dd, displays))

dashboard.layout.border = '1px grey dotted'
outleft.layout.width = "100%"
outleft.layout.height = "100%"
outleft.layout.border = "1px blue solid"
outright.layout.width = "100%"
outright.layout.height = "100%"
outright.layout.border = "1px green solid"

display(dashboard)